<a href="https://colab.research.google.com/github/gbr-guimaraes/Projeto-organizador-de-estudos/blob/main/Projeto_Cronograma_de_estudos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalação das bibliotecas

In [41]:
!pip install -q -U google-generativeai  # Instala a biblioteca google-generativeai
!pip install PyPDF2 -q                  # Instala a biblioteca PyPDF2

#Importação das dependencias

In [42]:
from google.colab import userdata
import google.generativeai as genai
import os
import PyPDF2
import ipywidgets as widgets
from IPython.display import display, Markdown
import textwrap

#Definição das variaveis de configuração

In [43]:
API_KEY = userdata.get('SECRET_KEY')  # Obtém a chave de API do Gemni
max_token = 1048576                   # Define o limite máximo de tokens para o modelo
list_command = "Apresente os cargos disponiveis neste edital em formato de lista simples,apenas texto, um elemento em cada linha, sem detalhes, textos de introdução ou caracteres especiais"
main_command = "A partir do texto acima, crie um cronograma de estudos personalizado, em formato de tabela Markdown, com os conteúdos gerais e específicos para o cargo de {role} para o estudante {student_name}, este aluno tem disponibilidade de estudar {hours} horas por dia, nos dias {week_days}"

#Configuração do modelo

In [44]:
genai.configure(api_key=API_KEY)          # Configura a API com a chave
model = genai.GenerativeModel('gemini-1.5-pro-latest')  # Define o modelo a ser usado

#Função para extrair o texto de um PDF em partes menores

In [45]:
def extract_pdf_pages(pathname: str) -> list[str]:
    parts = [f"--- START OF PDF ${pathname} ---"]       # Inicia a lista de partes com um cabeçalho
    with open(pathname, "rb") as pdf_file:              # Abre o arquivo PDF em modo binário
        pdf_reader = PyPDF2.PdfReader(pdf_file)         # Cria um objeto leitor de PDF
        num_pages = len(pdf_reader.pages)               # Obtém o número de páginas
        for page_num in range(num_pages):               # Itera sobre cada página
            page = pdf_reader.pages[page_num]           # Obtém a página atual
            page_text = page.extract_text()             # Extrai o texto da página
            if len(parts) + len(page_text) > max_token: # Verifica se a parte excede o limite de tokens
                # Divide a entrada em partes menores
                convo.send_message(parts)
                parts = []
            parts.append(f"--- PAGE {page_num} ---")    # Adiciona um cabeçalho para a página
            parts.append(page_text)                     # Adiciona o texto da página à lista de partes
    return parts

# Função para obter a lista de cargos disponíveis no edital

In [61]:
def role_list_generate():
    convo = model.start_chat(history=[])    # Inicia uma conversa com o modelo
    # Combina o texto do PDF com o comando para listar os cargos em uma única mensagem
    combined_message = "".join(extract_pdf_pages("/content/edital.pdf")) + list_command
    convo.send_message(combined_message)    # Envia a mensagem combinada ao modelo
    return convo.last.text

# Função para gerar um cronograma de estudos personalizado

In [64]:
def call_gemni(role: str, student_name: str, week_days: str, hours):
    convo = model.start_chat(history=[])
    combined_message = "".join(extract_pdf_pages("/content/edital.pdf")) + "\n\n" + main_command.format(role=role, student_name=student_name, hours=hours, week_days=week_days)
    convo.send_message(combined_message)
    display(to_markdown(convo.last.text))                                      # Imprime o cronograma gerado pelo modelo

# Função para atualizar a lista de cargos disponíveis após o upload do arquivo

In [48]:
def handle_file_upload(change):
    uploaded_file = change['new']
    file_save()
    roles_list = []
    roles_list = role_list_generate().splitlines() # Obtém a lista de cargos do modelo
    roles.options = roles_list                     # Atualiza as opções do widget de seleção de cargos

# Função para processar o clique no botão "Gerar"

In [49]:
def on_button_clicked(b):
    # Obtém os dias da semana selecionados pelo usuário
    checkboxes = [monday, tuesday, wednesday, thursday, friday, saturday, sunday]
    week_list = ""
    for checkbox in checkboxes:
        if checkbox.value:
            week_list += checkbox.description + ", "
    # Chama a função para gerar o cronograma com as informações do usuário
    call_gemni(roles.value, name.value, week_list, time_avaliable.value)

# Função para salvar o arquivo PDF carregado pelo usuário

In [50]:
def file_save():
    if file_upload.data != []:
      with open("./edital.pdf", "wb") as fp:
          fp.write(file_upload.data[0])

# Função para formatar a resposta da IA


In [51]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Criação dos widgets da interface do usuário

In [62]:
# Widget para upload do arquivo PDF
file_upload = widgets.FileUpload(accept='.pdf',multiple=False)
# Widgets para seleção dos dias da semana disponíveis para estudo
monday = widgets.Checkbox(value=False, description='Segunda-Feira', disabled=False, indent=False)
tuesday = widgets.Checkbox(value=False, description='Terça-Feira', disabled=False, indent=False)
wednesday = widgets.Checkbox(value=False, description='Quarta-Feira', disabled=False, indent=False)
thursday = widgets.Checkbox(value=False, description='Quinta-Feira', disabled=False, indent=False)
friday = widgets.Checkbox(value=False, description='Sexta-Feira', disabled=False, indent=False)
saturday = widgets.Checkbox(value=False, description='Sábado', disabled=False, indent=False)
sunday = widgets.Checkbox(value=False, description='Domingo', disabled=False, indent=False)
# Widget para definir o número de horas disponíveis por dia
time_avaliable = widgets.BoundedIntText(value=5, min=1, max=24, step=1, description='',disabled=False)
# Widget para inserir o nome do usuário
name = widgets.Text(value='', placeholder='Nome...', description='', disabled=False)
# Widget para selecionar o cargo desejado
roles = widgets.Dropdown(options=[], description='', disabled=False,)
# Botão "Gerar"
submit = widgets.ToggleButton(value=False, description='Gerar', disabled=False, button_style='', tooltip='Description', icon='check')

# Exibição dos widgets na interface e eventos

In [65]:
print("Faça o upload do seu edital:")
display(file_upload)
print("\nSeu Nome:")
display(name)
print("\nCargo desejado:")
display(roles)
print("\nDias disponiveis para estudo:")
display(monday, tuesday, wednesday, thursday, friday, saturday, sunday)
print('\nHoras disponiveis por dia:')
display(time_avaliable)
print("\n")
display(submit)
print("\n\n")

# Vincula as funções aos eventos dos widgets
file_upload.observe(handle_file_upload, names='value') # Chama a função quando um arquivo é carregado
submit.observe(on_button_clicked)                      # Chama a função quando o botão é clicado

Faça o upload do seu edital:


FileUpload(value={'edital.pdf': {'metadata': {'name': 'edital.pdf', 'type': 'application/pdf', 'size': 985568,…


Seu Nome:


Text(value='Gabriel', placeholder='Nome...')


Cargo desejado:


...

Checkbox(value=False, description='Domingo', indent=False)


Horas disponiveis por dia:


BoundedIntText(value=5, max=24, min=1)

ToggleButton(value=False, description='Gerar', icon='check', tooltip='Description')

> ## Cronograma de Estudos Personalizado para Gabriel - Concurso ATI (5h/dia)
> 
> **Objetivo:** Cargo de Analista em Tecnologia da Informação (ATI) - MGI
> 
> **Disponibilidade:** 5 horas por dia - Seg, Ter, Qui, Sex, Sáb (25h semanais)
> 
> **Período:** 12 Semanas (06/02/2024 - 28/04/2024) - **Ajustável conforme necessidade**
> 
> **Recursos:** Materiais de estudo específicos para o concurso ATI (apostilas, cursos online, etc.)
> 
> **Observações:**
> 
> * Este cronograma é um guia inicial e **deve ser adaptado** às suas necessidades e ritmo de aprendizado.
> * **Priorize os conteúdos com maior peso** na prova, conforme Anexo V do Edital.
> * **Revise os conteúdos já estudados** periodicamente para melhor fixação.
> * **Faça simulados** para avaliar seu progresso e identificar pontos fracos.
> * **Reserve tempo para descanso e lazer**, essenciais para manter o foco e a motivação.
> * **Mantenha-se atualizado sobre o concurso** no site oficial.
> 
> **Legenda:**
> 
> * CG: Conhecimentos Gerais
> * E1: Eixo Temático 1
> * E2: Eixo Temático 2
> * E3: Eixo Temático 3
> * E4: Eixo Temático 4
> * E5: Eixo Temático 5
> * Rev: Revisão
> 
> | Semana | Segunda-feira (5h) | Terça-feira (5h) | Quinta-feira (5h) | Sexta-feira (5h) | Sábado (5h) |
> |---|---|---|---|---|---|
> | **1 (06/02 - 10/02)** | CG: Políticas Públicas | CG: Desafios do Estado de Direito | CG: Ética e Integridade | CG: Diversidade e Inclusão | CG: Administração Pública Federal |
> | **2 (12/02 - 17/02)** | CG: Finanças Públicas | E1: Planejamento e Gestão Estratégica | E1: Gestão de Projetos | E1: Gestão de Processos | Rev: CG + E1 (exercícios) |
> | **3 (19/02 - 24/02)** | E2: O Processo de Elaboração de Políticas | E2: Implementação de Políticas | E2: Avaliação de Políticas | E2:  Políticas de CT&I | Rev: E2 (exercícios) |
> | **4 (26/02 - 02/03)** | E3: Segurança da Informação | E3: Governança e Gestão de TI | E3: Sistemas Operacionais | E3: Redes de Computadores | Rev: E3 (exercícios) |
> | **5 (04/03 - 09/03)** | E4: Programação e Estruturas de Dados | E4: Programação Web | E4: Desenvolvimento Móvel | E4: Engenharia de Software | Rev: E4 (exercícios) |
> | **6 (11/03 - 16/03)** | E5: Big Data | E5: Data Warehouse | E5: Descoberta de Conhecimento | E5: Aprendizado de Máquina | Rev: E5 (exercícios) |
> | **7 (18/03 - 23/03)** | E5: Estatística | E5: Probabilidade | E5: Inferência Estatística | E5: Amostragem | Simulado Geral 1 |
> | **8 (25/03 - 30/03)** | Rev: CG (pontos fracos) | Rev: E1 (pontos fracos) | Rev: E2 (pontos fracos) | Rev: E3 (pontos fracos) | Rev: E4 (pontos fracos) |
> | **9 (01/04 - 06/04)** | Rev: E5 (pontos fracos) | E4: Banco de Dados | E4: Arquitetura de Sistemas | E1: Compras Governamentais | E1: Organização da Adm. Pública | 
> | **10 (08/04 - 13/04)** | E2: Políticas de Governo Digital | Rev: CG + E1 (exercícios) | Rev: E2 + E3 (exercícios) | Rev: E4 + E5 (exercícios) | Simulado Geral 2 |
> | **11 (15/04 - 20/04)** | Rev: Pontos Fracos Simulado 2 | Rev: Legislação Específica | Rev: Conteúdos com Maior Peso | Simulado Geral 3 | Descanso |
> | **12 (22/04 - 28/04)** | Revisão Geral | Revisão Geral | Revisão Geral | Revisão Geral | Descanso | 
> 
> **Ajustes:**
> 
> * Ajuste a duração de cada semana e o tempo dedicado a cada conteúdo conforme sua necessidade.
> * Inclua revisões extras e simulados adicionais se achar necessário.
> 
> **Lembre-se:**  A disciplina e a persistência são seus maiores aliados! Boa sorte nos estudos, Gabriel! 🚀
